### SSA(Sample code)

In [ ]:
import numpy as np

class SparrowSearchAlgorithm:
    def __init__(self, obj_func, dim, lb, ub, max_iter=100, pop_size=30, PD_ratio=0.2, SD_ratio=0.1):
        self.obj_func = obj_func
        self.dim = dim
        self.lb = lb
        self.ub = ub
        self.max_iter = max_iter
        self.pop_size = pop_size
        self.PD = int(pop_size * PD_ratio)  # 发现者数量[7,8](@ref)
        self.SD = int(pop_size * SD_ratio)  # 警戒者数量
        self.ST = 0.6                        # 安全阈值
        
    def optimize(self):
        # 初始化种群
        X = np.random.uniform(self.lb, self.ub, (self.pop_size, self.dim))
        fitness = np.array([self.obj_func(x) for x in X])
        gbest = X[np.argmin(fitness)]
        gbest_fit = np.min(fitness)
        convergence = []
        
        for t in range(self.max_iter):
            # 发现者更新（Eq.3）
            R2 = np.random.rand()
            for i in range(self.PD):
                if R2 < self.ST:
                    X[i] *= np.exp(-i / (0.5 * self.max_iter))  # 指数衰减搜索[8](@ref)
                else:
                    X[i] += np.random.randn(self.dim) * 0.1      # 随机扰动
            
            # 加入者更新（Eq.4）
            A = np.random.choice([-1, 1], self.dim)
            for i in range(self.PD, self.pop_size):
                if i > (self.pop_size + self.PD) // 2:
                    X[i] = np.random.randn(self.dim) * np.exp((gbest - X[i]) / (i+1)**2)
                else:
                    X[i] = gbest + np.abs(X[i] - gbest).dot(A) * (A / (A @ A.T))
            
            # 警戒者更新（Eq.5）
            danger_idx = np.random.choice(self.pop_size, self.SD, replace=False)
            for j in danger_idx:
                if fitness[j] > gbest_fit:
                    X[j] = gbest + np.random.randn(self.dim) * np.abs(X[j] - gbest)
                else:
                    K = 2 * np.random.rand() - 1
                    X[j] += K * (np.abs(X[j] - X[-1]) / (fitness[j] - fitness[-1] + 1e-8))
            
            # 边界约束与适应度更新
            X = np.clip(X, self.lb, self.ub)
            fitness = np.array([self.obj_func(x) for x in X])
            
            # 更新全局最优
            if (current_min := np.min(fitness)) < gbest_fit:
                gbest = X[np.argmin(fitness)]
                gbest_fit = current_min
            convergence.append(gbest_fit)
        
        return gbest, gbest_fit, convergence

# 示例调用
def sphere(x):
    return sum(x**2)

ssa = SparrowSearchAlgorithm(sphere, dim=3, lb=[-5,-5,-5], ub=[5,5,5])
best_sol, best_fit, curve = ssa.optimize()
print(f"SSA最优解：{best_sol.round(4)}, 适应度：{best_fit:.6f}")